<a href="https://colab.research.google.com/github/junaina/EmpathAI/blob/wip/EmpathAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!git clone https://github.com/junaina/EmpathAI.git
%cd EmpathAI

Cloning into 'EmpathAI'...
/content/EmpathAI


In [14]:
# set your name/email so commits look right
!git config --global user.email "i222633@nu.edu.pk"
!git config --global user.name  "junaina"

# point “origin” at the repo over HTTPS, embedding your token
!git remote set-url origin https://$GITHUB_TOKEN@github.com/junaina/EmpathAI.git

# make sure we’re on main (rename if needed) and push it to create it upstream
!git branch -M main
!git push -u origin main


Enumerating objects: 3, done.
Counting objects: 100% (3/3), done.
Writing objects: 100% (3/3), 217 bytes | 217.00 KiB/s, done.
Total 3 (delta 0), reused 0 (delta 0), pack-reused 0
To https://github.com/junaina/EmpathAI.git
 * [new branch]      main -> main
Branch 'main' set up to track remote branch 'main' from 'origin'.


In [15]:
!git checkout -b wip
!git push -u origin wip

Switched to a new branch 'wip'
Total 0 (delta 0), reused 0 (delta 0), pack-reused 0
remote: 
remote: Create a pull request for 'wip' on GitHub by visiting:
remote:      https://github.com/junaina/EmpathAI/pull/new/wip
remote: 
To https://github.com/junaina/EmpathAI.git
 * [new branch]      wip -> wip
Branch 'wip' set up to track remote branch 'wip' from 'origin'.


In [16]:
# make sure you’ve already set your GITHUB_TOKEN and git config
!git clone https://$GITHUB_TOKEN@github.com/junaina/EmpathAI.git
%cd EmpathAI

Cloning into 'EmpathAI'...
remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Total 3 (delta 0), reused 3 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (3/3), done.
/content/EmpathAI/EmpathAI
